In [2]:
import xgboost as xgb
print(xgboost.__version__)

0.90


In [5]:
from xgboost import plot_importance
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv('./EGR_Cooler_data/train_data.csv')
test = pd.read_csv('./EGR_Cooler_data/test_data.csv')

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 70 columns):
body_length                8000 non-null float64
body_width                 8000 non-null float64
body_height                8000 non-null float64
wavyfin_length             8000 non-null float64
wavyfin_width              8000 non-null float64
wavyfin_height             8000 non-null float64
pipe1_length               8000 non-null float64
pipe1_width                8000 non-null float64
pipe1_height               8000 non-null float64
pipe2_length               8000 non-null float64
pipe2_width                8000 non-null float64
pipe2_height               8000 non-null float64
flange1_length             8000 non-null float64
flange1_width              8000 non-null float64
flange1_height             8000 non-null float64
flange2_length             8000 non-null float64
flange2_width              8000 non-null float64
flange2_height             8000 non-null float64
op10_length

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 70 columns):
body_length                2000 non-null float64
body_width                 2000 non-null float64
body_height                2000 non-null float64
wavyfin_length             2000 non-null float64
wavyfin_width              2000 non-null float64
wavyfin_height             2000 non-null float64
pipe1_length               2000 non-null float64
pipe1_width                2000 non-null float64
pipe1_height               2000 non-null float64
pipe2_length               2000 non-null float64
pipe2_width                2000 non-null float64
pipe2_height               2000 non-null float64
flange1_length             2000 non-null float64
flange1_width              2000 non-null float64
flange1_height             2000 non-null float64
flange2_length             2000 non-null float64
flange2_width              2000 non-null float64
flange2_height             2000 non-null float64
op10_length

In [9]:
X_train = train.drop(['op10_test', 'op10_time_series', 'op20_test', 'op20_time_series',
                            'op30_test', 'op30_time_series', 'op40_test', 'op40_time_series', 
                            'op50_test', 'op50_time_series', 'op60_test', 
                            'op40_length', 'op40_width', 'op40_height', 
                            'op40_light_sensor', 'op40_temperature', 'op40_scratch', 'op40_operator', 
                            'op50_length', 'op50_width', 'op50_height',
                            'op50_light_sensor', 'op50_temperature', 'op50_scratch', 'op50_operator',
                            'op60_length', 'op60_width', 'op60_height', 
                            'op60_light_sensor', 'op60_scratch', 'uniform_distribution'], axis = 1)
X_train.head()

,body_length,body_width,body_height,wavyfin_length,wavyfin_width,wavyfin_height,pipe1_length,pipe1_width,pipe1_height,pipe2_length,...,op20_light_sensor,op20_scratch,op20_operator,op30_length,op30_width,op30_height,op30_electricity_sensor,op30_light_sensor,op30_scratch,op30_operator
0,199.996,100.002,50.001,100.000,49.997,59.997,30.000,50.003,29.996,30.000,...,0,1,1,199.995873,180.106820,59.996905,94.711471,0,1,1
1,200.002,100.000,49.998,99.999,50.002,60.000,29.999,49.994,29.999,30.007,...,0,1,1,200.001735,179.993792,59.999933,94.880748,0,1,1
2,199.998,100.000,49.998,100.003,50.003,59.998,29.995,50.002,30.000,30.003,...,0,1,1,199.997595,180.007309,59.997682,92.783317,0,1,1
3,200.002,100.000,50.000,100.000,50.003,60.000,30.001,49.995,29.998,30.002,...,0,1,1,200.001981,179.995833,60.000453,93.951689,0,1,1
4,200.001,100.001,49.999,100.003,49.997,59.996,30.000,50.001,29.996,30.002,...,0,1,1,200.001256,180.005777,59.995941,90.609768,0,1,1


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 39 columns):
body_length                8000 non-null float64
body_width                 8000 non-null float64
body_height                8000 non-null float64
wavyfin_length             8000 non-null float64
wavyfin_width              8000 non-null float64
wavyfin_height             8000 non-null float64
pipe1_length               8000 non-null float64
pipe1_width                8000 non-null float64
pipe1_height               8000 non-null float64
pipe2_length               8000 non-null float64
pipe2_width                8000 non-null float64
pipe2_height               8000 non-null float64
flange1_length             8000 non-null float64
flange1_width              8000 non-null float64
flange1_height             8000 non-null float64
flange2_length             8000 non-null float64
flange2_width              8000 non-null float64
flange2_height             8000 non-null float64
op10_length

In [11]:
Y_train = train['op60_test']
Y_train.head()

0    1
1    0
2    0
3    0
4    0
Name: op60_test, dtype: int64

In [12]:
X_test = test.drop(['op10_test', 'op10_time_series', 'op20_test', 'op20_time_series',
                            'op30_test', 'op30_time_series', 'op40_test', 'op40_time_series', 
                            'op50_test', 'op50_time_series', 'op60_test', 
                            'op40_length', 'op40_width', 'op40_height', 
                            'op40_light_sensor', 'op40_temperature', 'op40_scratch', 'op40_operator', 
                            'op50_length', 'op50_width', 'op50_height',
                            'op50_light_sensor', 'op50_temperature', 'op50_scratch', 'op50_operator',
                            'op60_length', 'op60_width', 'op60_height', 
                            'op60_light_sensor', 'op60_scratch', 'uniform_distribution'], axis = 1)
X_test.head()

,body_length,body_width,body_height,wavyfin_length,wavyfin_width,wavyfin_height,pipe1_length,pipe1_width,pipe1_height,pipe2_length,...,op20_light_sensor,op20_scratch,op20_operator,op30_length,op30_width,op30_height,op30_electricity_sensor,op30_light_sensor,op30_scratch,op30_operator
0,199.997,99.999,50.003,100.000,50.005,60.003,29.996,50.004,30.002,29.998,...,0,1,3,199.997061,180.005558,60.002786,95.000563,0,1,3
1,199.999,100.001,50.002,99.996,50.000,60.004,29.998,50.004,29.995,30.000,...,0,1,3,199.999360,180.005180,60.003705,91.815773,0,1,3
2,199.999,100.001,50.002,100.001,50.001,60.003,29.997,50.001,30.003,30.001,...,0,1,3,199.999315,180.004519,60.002609,94.368763,0,1,3
3,200.002,99.996,49.999,100.001,49.998,60.000,30.000,50.006,30.001,30.001,...,0,1,3,200.002048,180.001510,59.999508,96.679591,0,1,3
4,199.999,100.003,50.003,100.000,50.001,60.000,30.000,49.998,30.002,30.001,...,0,1,3,199.999422,180.001309,59.999977,94.037526,0,1,3


In [13]:
Y_test = test['op60_test']
Y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: op60_test, dtype: int64

In [14]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 39 columns):
body_length                2000 non-null float64
body_width                 2000 non-null float64
body_height                2000 non-null float64
wavyfin_length             2000 non-null float64
wavyfin_width              2000 non-null float64
wavyfin_height             2000 non-null float64
pipe1_length               2000 non-null float64
pipe1_width                2000 non-null float64
pipe1_height               2000 non-null float64
pipe2_length               2000 non-null float64
pipe2_width                2000 non-null float64
pipe2_height               2000 non-null float64
flange1_length             2000 non-null float64
flange1_width              2000 non-null float64
flange1_height             2000 non-null float64
flange2_length             2000 non-null float64
flange2_width              2000 non-null float64
flange2_height             2000 non-null float64
op10_length

In [31]:
print(train['op60_test'].value_counts())

0    7466
1     534
Name: op60_test, dtype: int64


In [32]:
print(test['op60_test'].value_counts())

0    1881
1     119
Name: op60_test, dtype: int64


In [16]:
dtrain = xgb.DMatrix(data = X_train, label = Y_train)
dtest = xgb.DMatrix(data = X_test, label = Y_test)

In [121]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]

In [171]:
params = {'max_depth' : 5,
         'eta': 0.0001,
         'objective' : 'binary:logistic',
          'eval_metric' : 'error',
         'early_stopping':200
         }
num_rounds = 1000

In [172]:
xgb_model = xgb.train(params = params, dtrain = dtrain, num_boost_round = num_rounds,
                     early_stopping_rounds = 200, evals = wlist)

[0]	train-error:0.01925	eval-error:0.022
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 200 rounds.
[1]	train-error:0.01925	eval-error:0.022
[2]	train-error:0.01925	eval-error:0.022
[3]	train-error:0.01925	eval-error:0.022
[4]	train-error:0.01925	eval-error:0.022
[5]	train-error:0.01925	eval-error:0.022
[6]	train-error:0.01925	eval-error:0.022
[7]	train-error:0.01925	eval-error:0.022
[8]	train-error:0.01925	eval-error:0.022
[9]	train-error:0.01925	eval-error:0.022
[10]	train-error:0.01925	eval-error:0.022
[11]	train-error:0.01925	eval-error:0.022
[12]	train-error:0.01925	eval-error:0.022
[13]	train-error:0.01925	eval-error:0.022
[14]	train-error:0.01925	eval-error:0.022
[15]	train-error:0.01925	eval-error:0.022
[16]	train-error:0.01925	eval-error:0.022
[17]	train-error:0.01925	eval-error:0.022
[18]	train-error:0.01925	eval-error:0.022
[19]	train-error:0.01925	eval-error:0.022
[20]	train-error:0.01925

[188]	train-error:0.01875	eval-error:0.0185
[189]	train-error:0.01875	eval-error:0.0185
[190]	train-error:0.01875	eval-error:0.0185
[191]	train-error:0.01875	eval-error:0.0185
[192]	train-error:0.01875	eval-error:0.0185
[193]	train-error:0.01875	eval-error:0.0185
[194]	train-error:0.01875	eval-error:0.0185
[195]	train-error:0.01875	eval-error:0.0185
[196]	train-error:0.01875	eval-error:0.0185
[197]	train-error:0.01875	eval-error:0.0185
[198]	train-error:0.01875	eval-error:0.0185
[199]	train-error:0.01875	eval-error:0.0185
[200]	train-error:0.01875	eval-error:0.0185
[201]	train-error:0.01875	eval-error:0.0185
[202]	train-error:0.01875	eval-error:0.0185
[203]	train-error:0.01875	eval-error:0.0185
[204]	train-error:0.01875	eval-error:0.0185
[205]	train-error:0.01875	eval-error:0.0185
[206]	train-error:0.01875	eval-error:0.0185
[207]	train-error:0.01875	eval-error:0.0185
[208]	train-error:0.01875	eval-error:0.0185
[209]	train-error:0.01875	eval-error:0.0185
[210]	train-error:0.01875	eval-e

In [173]:
pred_probs = xgb_model.predict(dtest)
print(np.round(pred_probs[:10], 3))

print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨 ')
preds = [1 if x > 0.5 else 0 for x in pred_probs]
print('예측값 10개만 표시: ', preds[:100])

[0.486 0.486 0.486 0.486 0.486 0.486 0.486 0.486 0.486 0.486]
predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨 
예측값 10개만 표시:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [174]:
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy , precision ,recall))

In [175]:
get_clf_eval(Y_test, preds)

오차 행렬
[[1880    1]
 [  36   83]]
정확도: 0.9815, 정밀도: 0.9881, 재현율: 0.6975
